In [ ]:
import deeplay as dl
import torch.nn as nn

# minimal MLP

class MultiLayerPerceptron(dl.DeeplayModule):

    def __init__(self, in_features, hidden_features, out_features):
        super().__init__()
        
        in_features_per_layer = [in_features, *hidden_features]
        out_features_per_layer = [*hidden_features, out_features]

        self.blocks = dl.LayerList()
        for in_features, out_features in zip(in_features_per_layer, out_features_per_layer):
            self.blocks.append(
                dl.LayerActivationBlock(
                    dl.Layer(nn.Linear, in_features, out_features),
                    dl.Layer(nn.ReLU)
                )
            )

    def forward(self, x):
        for block in self.blocks:
            x = block(x)

        return x
    
mlp = MultiLayerPerceptron(2, [32, 32], 2)

In [ ]:
# Minimal classifier
import torchmetrics as tm

class Classifier(dl.Application):

    def __init__(self, model, **kwargs):
        self.model = model
        super().__init__(**kwargs)
        
        
    def forward(self, x):
        return self.model(x)
    
classifier = Classifier(
    mlp, 
    loss=nn.CrossEntropyLoss(), 
    optimizer=dl.Adam(lr=1e-3), 
    metrics=[tm.Accuracy("multiclass", num_classes=2)]
)

classifier.model.blocks[-1].activation.configure(nn.Identity)
classifier.build()

In [ ]:
# example:

import torch
n_samples = 10000
train_data = torch.rand(n_samples, 2)
train_labels = (train_data.sum(dim=1) > 1).long()

dataset = torch.utils.data.TensorDataset(train_data, train_labels)
train, val = torch.utils.data.random_split(dataset, [0.8, 0.2])

train_dataloader = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=16, shuffle=False)

trainer = dl.Trainer(max_epochs=10)

trainer.fit(classifier, train_dataloader, val_dataloader)
trainer.test(classifier, val_dataloader)